<a href="https://colab.research.google.com/github/arnav-meduri/JCVI-Syn3A-analysis/blob/main/FindBoundingBoxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install opencv-python
! pip install numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/.shortcut-targets-by-id/1Gd6Qvx67HvZr5HE48UMhRc2vhUfBl1Jh/PVO_Segmentation_Project/Annotated_PVO_OM_7-2-23/"
!ls -laF $folder_path

total 172031
-rw------- 1 root root 4939321 Jul  1 19:39 '216 OD- 1.png'
-rw------- 1 root root 4890773 Jul  1 19:39 '216 OD- 2.png'
-rw------- 1 root root 4948651 Jul  1 19:39 '216 OD- 3.png'
-rw------- 1 root root 4939316 Jul  1 19:39 '216 OD- 4.png'
-rw------- 1 root root 4820389 Jul  1 19:39 '216 OD- 5.png'
-rw------- 1 root root 3041556 Jul  1 15:50 '24 OS- 1 (1).png'
-rw------- 1 root root 3056103 Jul  1 15:51 '24 OS- 2.png'
-rw------- 1 root root 3023277 Jul  1 15:52 '24 OS- 3.png'
-rw------- 1 root root 3021458 Jul  3 01:34 '24 OS- 4.png'
-rw------- 1 root root 3027693 Jul  1 15:53 '24 OS- 5 (1).png'
-rw------- 1 root root 4826426 Jul  1 19:39 '298 OS-1.png'
-rw------- 1 root root 4778798 Jul  1 19:39 '298 OS-2.png'
-rw------- 1 root root 4835214 Jul  1 19:39 '298 OS-3.png'
-rw------- 1 root root 4807548 Jul  1 19:39 '298 OS-4.png'
-rw------- 1 root root 4768877 Jul  1 19:39 '298 OS-5.png'
-rw------- 1 root root 4869357 Jul  1 16:55 '357 OD-1.png'
-rw------- 1 root root 4729393

In [ ]:
import cv2
import numpy as np

def is_grayscale(img_path):
    # Load image
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

    if len(img.shape) == 3 and img.shape[2] == 4:
      img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    # If image is grayscale both dimensions of 3D image array will be equal
    if len(img.shape) < 3:
        return True
    if img.shape[2]  == 1:
        return True

    # If color image is read as grayscale, all channels will be equal.
    # Compare all channels to see if they are equal
    b,g,r = img[:,:,0], img[:,:,1], img[:,:,2]
    if (b==g).all() and (b==r).all():
        return True

    return False

In [ ]:
def findBoundingBox(img_path):

  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

  # Convert the image to HSV
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  # Define range for red color in HSV space
  lower_red1 = np.array([0,70,50])
  upper_red1 = np.array([10,255,255])
  lower_red2 = np.array([170,70,50])
  upper_red2 = np.array([180,255,255])

  # Threshold the HSV image to get only red colors
  mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
  mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
  mask = mask1 + mask2

  mask = cv2.erode(mask, None, iterations=2)
  mask = cv2.dilate(mask, None, iterations=2)

  # Find contours in the mask
  cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  coordinates = []

  # Iterate over the contours and find the bounding boxes
  for cnt in cnts:
      x,y,w,h = cv2.boundingRect(cnt)
      coordinates.append(((x, y), (x+w, y+h)))

  return coordinates


In [ ]:
import os
for file in os.listdir(folder_path):
  if file.endswith('.png'):
    full_path = os.path.join(folder_path, file)
    print(f'Examining file {file}')
    print(f'\tGrayScale? {is_grayscale(full_path)}')
    coordinates = findBoundingBox(full_path)
    print(f'\tNumber of bounding boxes={len(coordinates)}')
    for coord in coordinates:
      print(f'\ttop left: {coord[0]}, bottom right: {coord[1]}')


Examining file 24 OS- 1 (1).png
	GrayScale? False
	Number of bounding boxes=3
	top left: (766, 241), bottom right: (794, 276)
	top left: (1039, 91), bottom right: (1082, 135)
	top left: (668, 88), bottom right: (693, 117)
Examining file 24 OS- 2.png
	GrayScale? False
	Number of bounding boxes=1
	top left: (779, 295), bottom right: (806, 331)
Examining file 24 OS- 3.png
	GrayScale? False
	Number of bounding boxes=1
	top left: (1221, 84), bottom right: (1247, 114)
Examining file 24 OS- 5 (1).png
	GrayScale? False
	Number of bounding boxes=3
	top left: (976, 263), bottom right: (998, 290)
	top left: (940, 244), bottom right: (961, 272)
	top left: (166, 111), bottom right: (186, 134)
Examining file 45 OS-5.png
	GrayScale? False
	Number of bounding boxes=14
	top left: (54, 365), bottom right: (83, 396)
	top left: (269, 351), bottom right: (298, 382)
	top left: (580, 346), bottom right: (612, 380)
	top left: (74, 319), bottom right: (127, 362)
	top left: (110, 267), bottom right: (138, 297)
